In [50]:
# !pip install pyTigerGraph
# !pip install -q jupyter-dash
# !pip install python-dotenv
# !pip install dash-cytoscape
# !pip install dash-bootstrap-components

In [51]:
# Imports
import pandas as pd
import numpy as np
import pyTigerGraph as tg
import dash
from jupyter_dash import JupyterDash
from dash import html, dcc, dash_table
from dash.dependencies import Input, Output
import dash_cytoscape as cyto
import dash_bootstrap_components as dbc

### Open Connection to Graph

In [52]:
# Connection Params
hostname = 'https://leaguerecommender.i.tgcloud.io/'
username = 'tigergraph'
password = 'tigergraph'
graphname = 'ChampionRecommendation'

In [53]:
conn = tg.TigerGraphConnection(host=hostname, username=username, password=password, graphname=graphname)
conn.apiToken = conn.getToken(conn.createSecret())

print("Connected!") 

Connected!


### Create Dashboard

In [54]:
app = JupyterDash(external_stylesheets=[dbc.themes.BOOTSTRAP])

In [55]:
# Get champion names
import json
champion_names = None
with open('out_data/champions.txt') as f:
    champion_names = json.load(f)

In [56]:
# Set different lanes
lanes = ['Top', 'Jg', 'Mid', 'Bot', 'Sup']

## Dropdown functions to update graph

#### Main Graph Functions

In [57]:
def getNetwork(main_champion):
  # Run query
  comms = conn.runInstalledQuery("GetAllRecs", params={"mainChampion": main_champion})[0]['@@edgeList']

  color_map = {'top_rec':'yellow', 'jg_rec': 'green', 'mid_rec': 'blue', 'bot_rec': 'red', 'sup_rec': 'purple'}
  vertices = {}
  els = []

  # Table args
  lanes = []
  distances = []
  champion_name = []
  
  for entry in comms:
    if 'reverse' in entry['e_type']:
      continue

    # Get to and from nodes
    source = entry['from_id']
    target = entry['to_id']

    # Add initial mainChampion
    if source not in vertices:
      if source == main_champion:
        els.append({'data': {'id': source, 'label': source}, 'classes': 'orange'})

    # Add our target if not already present
    if target not in vertices:
      els.append({'data': {'id': target, 'label': target}, 'classes':color_map[entry['e_type']]})

    # Add from and to
    els.append({'data': {'source': source, 'target': target}})
    
    # TABLE VALUES
    # Extract lane
    lane = entry['e_type'].split('_')[0].capitalize()
    
    # Add values to table args
    lanes.append(lane)
    dist = 1-entry['attributes']['dist']
    distances.append("{:0.4%}".format(dist))
    champion_name.append(target)
  
  # Create cytoscape network
  stylesheet = [{'selector': 'node',
                              'style': {
                              'content': 'data(label)'
                              }
                          }]
  
  for color in color_map.values():
    stylesheet.append({
                      'selector': f'.{color}',
                      'style': {
                          'background-color': color,
                        }
                      }
    )
  stylesheet.append({
    'selector': '.orange',
    'style': {
      'background-color': 'orange'
    }
  })

  network = cyto.Cytoscape(
                  id='cytoscape',
                  elements=els,
                  layout={'name': 'circle'},
                  stylesheet=stylesheet,
                style={'width': '100%', 'height': '500px', 'marginRight':'100px'}
              )

  # Create Table
  table_df = pd.DataFrame(data=(zip(champion_name, lanes, distances)), columns=['Champion', 'Lane', 'Similarity'])
  table_df = table_df.sort_values(['Lane', 'Similarity'], ascending=False)
  # Sort by list
  sorter = ['Top', 'Jg', 'Mid', 'Bot', 'Sup']
  table_df = table_df.set_index('Lane')
  table_df = table_df.loc[sorter, :].reset_index()
  
  table = html.Div(
    dash_table.DataTable(table_df.to_dict('records'), columns=[{"name": i, "id": i} for i in table_df.columns], id='champion-table',
                         cell_selectable=False),
    style={'overflowY': 'scroll'}
  )

  return network, table

In [58]:
def get_new_elements(main_champion: str):
    comms = conn.runInstalledQuery("GetAllRecs", params={"mainChampion": main_champion})[0]['@@edgeList']
    color_map = {'top_rec':'yellow', 'jg_rec': 'green', 'mid_rec': 'blue', 'bot_rec': 'red', 'sup_rec': 'purple'}
    vertices = {}
    els = []
    
    # Table args
    lanes = []
    distances = []
    champion_name = []
    
    for entry in comms:
        if 'reverse' in entry['e_type']:
            continue

        # Get to and from nodes
        source = entry['from_id']
        target = entry['to_id']

        # Add initial mainChampion
        if source not in vertices:
            if source == main_champion:
                els.append({'data': {'id': source, 'label': source}, 'classes': 'orange'})

        # Add our target if not already present
        if target not in vertices:
            els.append({'data': {'id': target, 'label': target}, 'classes':color_map[entry['e_type']]})

        # Add from and to
        els.append({'data': {'source': source, 'target': target}})
        
        # TABLE VALUES
        # Extract lane
        lane = entry['e_type'].split('_')[0].capitalize()
        
        # Add values to table args
        lanes.append(lane)
        dist = 1-entry['attributes']['dist']
        distances.append("{:0.4%}".format(dist))
        champion_name.append(target)
        
        
    table_df = pd.DataFrame(data=(zip(champion_name, lanes, distances)), columns=['Champion', 'Lane', 'Similarity'])
    table_df = table_df.sort_values(['Lane', 'Similarity'], ascending=False)
    # Sort by list
    sorter = ['Top', 'Jg', 'Mid', 'Bot', 'Sup']
    table_df = table_df.set_index('Lane')
    
    return els, table_df.loc[sorter, :].reset_index()

#### Callback functions

In [59]:
@app.callback(Output('cytoscape', 'layout'),
              Input('dropdown-update-layout', 'value'))
def update_layout(layout):
    return {
        'name': layout,
        'animate': True
    }

In [60]:
@app.callback(Output('cytoscape', 'elements'),
              Output('champion-table', 'data'),
              Input('dropdown-update-champion', 'value'))
def update_champion_graph(value):
    (els, table_df) = get_new_elements(value)
    return els, table_df.to_dict('records')

In [61]:
@app.callback(
    Output('champion-table', 'style_data_conditional'),
    Input('cytoscape', 'selectedNodeData'))
def highlightSelectedNodeInTable(data):
    if not data:
        return dash.no_update
    
    champ = [nodeData['label'] for nodeData in data][-1]
    return [{'if': {'filter_query': '{Champion} =' + champ},
            'backgroundColor': 'yellow'}]

### Lane Graph Functions

In [62]:
def getLaneNetwork(main_champion):
  comms = conn.runInstalledQuery("GetTopRecs", params={"mainChampion": main_champion})[0]['@@edgeList']

  color_map = {'top_rec':'yellow', 'jg_rec': 'green', 'mid_rec': 'blue', 'bot_rec': 'red', 'sup_rec': 'purple'}
  vertices = {}
  els = []
  
  for entry in comms:
    if 'reverse' in entry['e_type']:
      continue

    # Get to and from nodes
    source = entry['from_id']
    target = entry['to_id']

    # Add initial mainChampion
    if source not in vertices:
      if source == main_champion:
        els.append({'data': {'id': source, 'label': source}, 'classes': 'orange'})

    # Add our target if not already present
    if target not in vertices:
      els.append({'data': {'id': target, 'label': target}, 'classes':color_map[entry['e_type']]})

    # Add from and to
    els.append({'data': {'source': source, 'target': target}})
  
  stylesheet = [{'selector': 'node',
                              'style': {
                              'content': 'data(label)'
                              }
                          }]
  
  for color in color_map.values():
    stylesheet.append({
                      'selector': f'.{color}',
                      'style': {
                          'background-color': color,
                        }
                      }
    )
  stylesheet.append({
    'selector': '.orange',
    'style': {
      'background-color': 'orange'
    }
  })

  network = cyto.Cytoscape(
                  id='lanegraph',
                  elements=els,
                  layout={'name': 'circle'},
                  stylesheet=stylesheet,
                style={'width': '100%', 'height': '500px'}
              )

  return network

In [63]:
def get_new_lane_elements(main_champion: str, lane: str):
    comms = conn.runInstalledQuery(f"Get{lane}Recs", params={"mainChampion": main_champion})[0]['@@edgeList']
    color_map = {'top_rec':'yellow', 'jg_rec': 'green', 'mid_rec': 'blue', 'bot_rec': 'red', 'sup_rec': 'purple'}
    vertices = {}
    els = []
    
    for entry in comms:
        if 'reverse' in entry['e_type']:
            continue

        # Get to and from nodes
        source = entry['from_id']
        target = entry['to_id']

        # Add initial mainChampion
        if source not in vertices:
            if source == main_champion:
                els.append({'data': {'id': source, 'label': source}, 'classes': 'orange'})

        # Add our target if not already present
        if target not in vertices:
            els.append({'data': {'id': target, 'label': target}, 'classes':color_map[entry['e_type']]})

        # Add from and to
        els.append({'data': {'source': source, 'target': target}})
        
    return els

#### Callback functions

In [64]:
@app.callback(Output('lanegraph', 'elements'),
              Input('dropdown-update-champion', 'value'),
              Input('dropdown-update-lane', 'value'))
def update_lane_rec_champion(champion, lane):
    return get_new_lane_elements(champion, lane)

## Dash Elements

In [65]:
def title_card():
    return dbc.Card([
                dbc.CardBody([
                              html.Center(html.H1("League of Legends Champion Recommender", className='card-title')),
                            ])
              ],
              color='light', # Options include: primary, secondary, info, success, warning, danger, light, dark
              style={
                  "width":"98rem",
                  "marginLeft":"1rem",
                  "marginTop":"1rem",
                  "marginBottom":"1rem"
                  }
            )

### Table

### Main Graph Elements

In [66]:
def champion_recommendations_card():
    (network, table) = getNetwork('Aatrox')
    
    graph_layout_dropdown = dcc.Dropdown(id='dropdown-update-layout',
                                    value='circle',
                                    clearable=False,
                                    options=[
                                        {'label': name.capitalize(), 'value': name}
                                        for name in ['grid', 'random', 'circle', 'cose', 'concentric']
                                    ])
    
    network_card = dbc.Card([
            dbc.CardBody([
                html.H1("Champions to play.", className='card-title'),
                graph_layout_dropdown,
                network,
            ])
        ],
        outline=True,
        color='info',
        style={
            "width": "100%",
            "marginRight":"1rem",
            "marginTop":"1rem",
            "marginBottom":"1rem"
        }          
    )
    
    table_card = dbc.Card([
        dbc.CardBody([
            html.H1('Top 3 Champions per Lane', className='card-title'),
            html.P("Recommendations for other champions to play in every lane.", className='card-body'),
            table
        ])
    ],
    outline=True,
    color='info',
    style={
        "marginTop":"1rem",
        "marginBottom":"1rem",
        "marginLeft":"1rem",
        'maxwidth': '692px'
    }               
    )
    
    return network_card, table_card

### Lane Recs Graph

In [67]:
def champion_lane_recs_card():
    network = getLaneNetwork('Aatrox')
    dropdown = dcc.Dropdown(id='dropdown-update-lane',
                            value='Top',
                            options=lanes,
                            clearable=False,
                            )
    return dbc.Card([
            dbc.CardBody([
                html.H1("What champions per lane?", className='card-title'),
                dropdown,
                network
            ])
        ],
        outline=True,
        color='info',
        style={
            "width": "100%",
            "marginRight":"5rem",
            "marginTop":"1rem",
            "marginBottom":"1rem"
        }          
    )

In [68]:
def champion_select_dropdown():
    return dcc.Dropdown(id='dropdown-update-champion',
                        value='Aatrox',
                        options=champion_names,
                        clearable=False,
                        searchable=True
                        )

### Champions Who Recommend Card

In [69]:
def champsWhoRecStats(main_champion: str):
    comms = conn.runInstalledQuery("GetChampionsWhoRec", params={"mainChampion": main_champion})[0]['recs']
    
    count = 0
    
    lane_counts = {'Top': 0, 'Jg': 0, 'Mid': 0, 'Bot': 0, 'Sup': 0}
    
    for edge in comms:
        count += 1
        lane_counts[edge['attributes']['lane'].capitalize()] += 1
    
    sorted_lane_counts = sorted(lane_counts.items(), key=lambda lane: lane[1], reverse=True)
    highest_lane = sorted_lane_counts[0][0]
    if sorted_lane_counts[0][1] == 0:
        highest_lane = 'None'
    
    return count, highest_lane

In [70]:
def champions_who_rec_card():
    (counts, lane) = champsWhoRecStats('Aatrox')
    return dbc.Card([
        dbc.CardBody([
            html.H1("Champions who recommend Aatrox", id='who-recommends-title', className='card-title'),
            dbc.Row([
                dbc.Col([html.P(f'Number who recommend me: {counts}', id='num-who-recommend')],width=6),
                dbc.Col([html.P(f'Recommended the most by: {lane}', id='most-recommended-by')],width=6)
            ])
        ])
    ],
    outline=True,
    color='info',
    style={
        "width": "100%",
        "marginLeft":"1rem",
        'maxWidth': '692px',
        "marginTop":"1rem",
        "marginBottom":"1rem"
    }  
)

### Callback functions

In [71]:
@app.callback(Output('num-who-recommend', 'children'),
              Output('most-recommended-by', 'children'),
              Output('who-recommends-title', 'children'),
              Input('dropdown-update-champion', 'value'))
def updateChampionsWhoRec(main_champion: str):
    (count, lane) = champsWhoRecStats(main_champion)
    return f'Number who recommend me: {count}', f'Recommended the most by: {lane}', f"Champions who recommend {main_champion}"

In [72]:
network_card, table_card = champion_recommendations_card()

In [73]:
app.layout = html.Div(
    children=[
            dbc.Row(children=[
            title_card(),
            champion_select_dropdown()
    ], justify='center'), 
            dbc.Row([
                dbc.Col([
                        champions_who_rec_card(),
                        table_card
                        ], width=6), 
                dbc.Col([network_card, 
                         champion_lane_recs_card()
                         ], width=6)])]
)

app.run_server(mode='external', debug=True)

Dash app running on http://127.0.0.1:8050/
